In [3]:
import os

In [21]:
# The name of the PDF file the user is providing.
working_dir = os.path.abspath('')
pdf_file_name = 'Invoice_6.pdf'
#pdf_file_name = 'invoice_AG42923.pdf'

base_file_name = pdf_file_name[:-4]  # Slice the string, excluding the last 4 characters ('.pdf')

# The directory containing the PDF file.
pdf_dir = os.path.join(working_dir, 'pdfs')

# The file path to the PDF file for loading.
pdf_path = os.path.join(pdf_dir, pdf_file_name)

In [22]:
"""
This code sample shows Custom Extraction Model operations with the Azure Form Recognizer client library. 
The async versions of the samples require Python 3.6 or later.

To learn more, please visit the documentation - Quickstart: Form Recognizer Python client library SDKs
https://learn.microsoft.com/azure/applied-ai-services/form-recognizer/quickstarts/get-started-v3-sdk-rest-api?view=doc-intel-3.1.0&pivots=programming-language-python
"""

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

"""
Remember to remove the key from your code when you're done, and never post it publicly. For production, use
secure methods to store and access your credentials. For more information, see 
https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
"""
endpoint = "https://di-y7us5axonhst4.cognitiveservices.azure.com/"
key = "41b3897811734dc983407b2971811f61"

model_id = "invoices-1.1.0"
formUrl = os.path.join(pdf_dir, pdf_file_name)

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# # Make sure your document's type is included in the list of document types the custom model can analyze
# poller = document_analysis_client.begin_analyze_document_from_url(model_id, formUrl)
# result = poller.result()


with open(formUrl, "rb") as document:
#with open("Invoice_INV4393421.pdf", "rb") as document:
    #model_id ="prebuilt-invoice"
    #model_id = "cpinvoice"  # Replace with your model ID

    poller = document_analysis_client.begin_analyze_document(model_id=model_id, document=document)
    result = poller.result()

In [23]:
for idx, document in enumerate(result.documents):
    print("--------Analyzing document #{}--------".format(idx + 1))
    print("Document has type {}".format(document.doc_type))
    print("Document has confidence {}".format(document.confidence))
    print("Document was analyzed by model with ID {}".format(result.model_id))
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        print("......found field of type '{}' with value '{}' and with confidence {}".format(field.value_type, field_value, field.confidence))



--------Analyzing document #1--------
Document has type invoices-1.1.0:invoices-1.1.0
Document has confidence 0.995
Document was analyzed by model with ID invoices-1.1.0
......found field of type 'signature' with value 'unsigned' and with confidence 0.995
......found field of type 'string' with value '64.38' and with confidence 0.969
......found field of type 'string' with value '2/27/2024' and with confidence 0.995
......found field of type 'signature' with value 'unsigned' and with confidence 0.995
......found field of type 'signature' with value 'signed' and with confidence 0.995
......found field of type 'string' with value 'MADE Apps' and with confidence 0.995
......found field of type 'string' with value 'None' and with confidence 0.935
......found field of type 'string' with value '2/27/2024' and with confidence 0.995
......found field of type 'string' with value '10.5' and with confidence 0.963
......found field of type 'string' with value 'Kingdom Street London W2 6BD United K

In [24]:
# iterate over tables, lines, and selection marks on each page
for page in result.pages:
    print("\nLines found on page {}".format(page.page_number))
    for line in page.lines:
        print("...Line '{}'".format(line.content.encode('utf-8')))
    for word in page.words:
        print(
            "...Word '{}' has a confidence of {}".format(
                word.content.encode('utf-8'), word.confidence
            )
        )
    for selection_mark in page.selection_marks:
        print(
            "...Selection mark is '{}' and has a confidence of {}".format(
                selection_mark.state, selection_mark.confidence
            )
        )


Lines found on page 1
...Line 'b'CONTOSO''
...Line 'b'Innovation drives progress''
...Line 'b'111st Avenue''
...Line 'b'Redmond, WA 67891''
...Line 'b'(201) 555-0101''
...Line 'b'(201) 555-0102''
...Line 'b'INVOICE''
...Line 'b'CUSTOMER''
...Line 'b'MADE Apps''
...Line 'b'Kingdom Street''
...Line 'b'London''
...Line 'b'W2 6BD''
...Line 'b'United Kingdom''
...Line 'b'ISSUED: 2/27/2024''
...Line 'b'PRODUCT ID''
...Line 'b'UNIT PRICE''
...Line 'b'QUANTITY''
...Line 'b'TOTAL PRICE''
...Line 'b'5-08-XX''
...Line 'b'9.50''
...Line 'b'1.0''
...Line 'b'9.50''
...Line 'b'5-09-XX''
...Line 'b'5.00''
...Line 'b'5.0''
...Line 'b'25.00''
...Line 'b'5-13-XX''
...Line 'b'11.50''
...Line 'b'1.0''
...Line 'b'11.50''
...Line 'b'5-14-XX''
...Line 'b'5.25''
...Line 'b'3.5''
...Line 'b'18.38''
...Line 'b'TOTAL''
...Line 'b'10.5''
...Line 'b'64.38''
...Line 'b'Distributor signature:''
...Line 'b'Date: 2/27/2024''
...Line 'b'Customer signature:''
...Line 'b'Date:''
...Word 'b'CONTOSO'' has a confidence of 0

In [25]:
for i, table in enumerate(result.tables):
    print("\nTable {} can be found on page:".format(i + 1))
    for region in table.bounding_regions:
        print("...{}".format(i + 1, region.page_number))
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
                cell.row_index, cell.column_index, cell.content.encode('utf-8')
            )
        )
print("-----------------------------------")


Table 1 can be found on page:
...1
...Cell[0][0] has content 'b'PRODUCT ID''
...Cell[0][1] has content 'b'UNIT PRICE''
...Cell[0][2] has content 'b'QUANTITY''
...Cell[0][3] has content 'b'TOTAL PRICE''
...Cell[1][0] has content 'b'5-08-XX''
...Cell[1][1] has content 'b'9.50''
...Cell[1][2] has content 'b'1.0''
...Cell[1][3] has content 'b'9.50''
...Cell[2][0] has content 'b'5-09-XX''
...Cell[2][1] has content 'b'5.00''
...Cell[2][2] has content 'b'5.0''
...Cell[2][3] has content 'b'25.00''
...Cell[3][0] has content 'b'5-13-XX''
...Cell[3][1] has content 'b'11.50''
...Cell[3][2] has content 'b'1.0''
...Cell[3][3] has content 'b'11.50''
...Cell[4][0] has content 'b'5-14-XX''
...Cell[4][1] has content 'b'5.25''
...Cell[4][2] has content 'b'3.5''
...Cell[4][3] has content 'b'18.38''
...Cell[5][0] has content 'b'TOTAL''
...Cell[5][1] has content 'b'''
...Cell[5][2] has content 'b'10.5''
...Cell[5][3] has content 'b'64.38''

Table 2 can be found on page:
...2
...Cell[0][0] has content 'b'Di